In [ ]:
#exploring the coarse and fine transformers to see if i understand it right
!rm -rf audiolm_pytorch/ setup.py audiolm-pytorch.zip # clean out any old stuff floating around

!pip install torch datasets boto3

# download audiolm_pytorch manually so i can inject print statements
# !pip uninstall -y audiolm_pytorch

raise AssertionError("don't forget to upload the customized version of audiolm_pytorch with print statements")

# !zip -r audiolm_pytorch.zip audiolm_pytorch/

import urllib.request
import os
import zipfile
if not os.path.isfile("audiolm-pytorch.zip"):
  urllib.request.urlretrieve("https://github.com/LWProgramming/audiolm-pytorch/archive/refs/heads/personal_hacks.zip", "audiolm-pytorch.zip")
if not os.path.isdir("audiolm-pytorch"):
  with zipfile.ZipFile("audiolm-pytorch.zip", 'r') as zip_ref:
    zip_ref.extractall("audiolm-pytorch")
!mv audiolm-pytorch/audiolm-pytorch-personal_hacks/audiolm_pytorch .

# install necessary files for patched audiolm-pytorch
!mv audiolm-pytorch/audiolm-pytorch-personal_hacks/setup.py .
!pip install . # install requirements from the patched audiolm-pytorch dir
!rm -rf audiolm-pytorch # not the one with underscore which is the actual library



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.5/754.5 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

%set_env AWS_SHARED_CREDENTIALS_FILE=drive/MyDrive/Colab Notebooks/AWS-SECRET-credentials
%set_env AWS_CONFIG_FILE=drive/MyDrive/Colab Notebooks/AWS-config

Mounted at /content/drive/
env: AWS_SHARED_CREDENTIALS_FILE=drive/MyDrive/Colab Notebooks/AWS-SECRET-credentials
env: AWS_CONFIG_FILE=drive/MyDrive/Colab Notebooks/AWS-config


In [ ]:
# # load soundstream checkpoint from version 0.11.1-- never mind, doesn't work
# import boto3

# boto3.setup_default_session(profile_name='mltraining')
# s3_resource = boto3.resource('s3')
# bucket_name = "itsleonwu-paperspace"
# bucket = s3_resource.Bucket(name=bucket_name)

# # for obj in bucket.objects.all():
# #     print(obj.key)
# soundstream_ckpt = "actual_soundstream_results/20000.pt" # name of file in bucket
# bucket.download_file(soundstream_ckpt, "soundstream_ckpt.pt")

In [ ]:
# # in case not already loaded
# from datasets import load_dataset
# # load demo audio and set processor
# dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")

# !rm -rf placeholder_dataset

import math
import wave
import struct
# Placeholder data generation
def get_sinewave(freq=440.0, duration_ms=200, volume=1.0, sample_rate=44100.0):
  # code adapted from https://stackoverflow.com/a/33913403
  audio = []
  num_samples = duration_ms * (sample_rate / 1000.0)
  for x in range(int(num_samples)):
    audio.append(volume * math.sin(2 * math.pi * freq * (x / sample_rate)))
  return audio

def save_wav(file_name, audio, sample_rate=44100.0):
  # Open up a wav file
  wav_file=wave.open(file_name,"w")
  # wav params
  nchannels = 1
  sampwidth = 2
  # 44100 is the industry standard sample rate - CD quality.  If you need to
  # save on file size you can adjust it downwards. The stanard for low quality
  # is 8000 or 8kHz.
  nframes = len(audio)
  comptype = "NONE"
  compname = "not compressed"
  wav_file.setparams((nchannels, sampwidth, sample_rate, nframes, comptype, compname))
  # WAV files here are using short, 16 bit, signed integers for the 
  # sample size.  So we multiply the floating point data we have by 32767, the
  # maximum value for a short integer.  NOTE: It is theortically possible to
  # use the floating point -1.0 to 1.0 data directly in a WAV file but not
  # obvious how to do that using the wave module in python.
  for sample in audio:
      wav_file.writeframes(struct.pack('h', int( sample * 32767.0 )))
  wav_file.close()
  return

def make_placeholder_dataset(dataset_folder="placeholder_dataset"):
  # Make a placeholder dataset with a few .wav files that you can "train" on, just to verify things work e2e
  if os.path.isdir(dataset_folder):
    return
  os.makedirs(dataset_folder)
  save_wav(f"{dataset_folder}/example.wav", get_sinewave(6000))
  save_wav(f"{dataset_folder}/example2.wav", get_sinewave(duration_ms=5000))
  os.makedirs(f"{dataset_folder}/subdirectory")
  save_wav(f"{dataset_folder}/subdirectory/example.wav", get_sinewave(freq=330.0))

make_placeholder_dataset()

In [ ]:
# original semantic transformer
import torch
from audiolm_pytorch import HubertWithKmeans, SemanticTransformer, SemanticTransformerWrapper, SemanticTransformerTrainer
import os
import urllib

# hubert checkpoints can be downloaded at
# https://github.com/facebookresearch/fairseq/tree/main/examples/hubert

hubert_ckpt = 'hubert/hubert_base_ls960.pt'
hubert_quantizer = f'hubert/hubert_base_ls960_L9_km500.bin' # listed in row "HuBERT Base (~95M params)", column Quantizer
if not os.path.isdir("hubert"):
  os.makedirs("hubert")
if not os.path.isfile(hubert_ckpt):
  hubert_ckpt_download = f"https://dl.fbaipublicfiles.com/{hubert_ckpt}"
  urllib.request.urlretrieve(hubert_ckpt_download, f"./{hubert_ckpt}")
if not os.path.isfile(hubert_quantizer):
  hubert_quantizer_download = f"https://dl.fbaipublicfiles.com/{hubert_quantizer}"
  urllib.request.urlretrieve(hubert_quantizer_download, f"./{hubert_quantizer}")

wav2vec = HubertWithKmeans(
    checkpoint_path = './hubert/hubert_base_ls960.pt',
    kmeans_path = './hubert/hubert_base_ls960_L9_km500.bin'
)

semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6
).cuda()

semantic = SemanticTransformerWrapper(
            wav2vec = wav2vec,
            transformer = semantic_transformer,
            audio_conditioner = None,
            unique_consecutive = True
        ).cuda()


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MiniBatchKMeans from version 0.24.0 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
import torch
from audiolm_pytorch import HubertWithKmeans, AudioLMSoundStream, CoarseTransformer, CoarseTransformerWrapper, CoarseTransformerTrainer

soundstream = AudioLMSoundStream(
    # rq_num_quantizers=8
)
# soundstream.load('soundstream_ckpt.pt')

coarse_transformer = CoarseTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    codebook_size = 1024,
    num_coarse_quantizers = 3,
    dim = 512,
    depth = 6
)

# coarse = CoarseTransformerWrapper(transformer=coarse_transformer, soundstream=soundstream, wav2vec=wav2vec)

trainer = CoarseTransformerTrainer(
    transformer = coarse_transformer,
    soundstream = soundstream,
    wav2vec = wav2vec,
    folder = 'placeholder_dataset',
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 3
)

trainer.train()

# TODO: continue from here.
# - why doesn't generate end up calling forward via forward_with_cond_scale?
# should then hit part where we get sem and coarse token id shapes

training with dataset of 2 samples and validating with randomly splitted 1 samples
do you want to clear previous experiment checkpoints and results? (y/n) y
embed.keys(): dict_keys(['x', 'padding_mask', 'features'])
embed['x'] shape: torch.Size([1, 31, 768]), embed['features'].shape: torch.Size([1, 31, 768])
wav_input shape: torch.Size([1, 10240]), embed shape: torch.Size([31, 768]), packed_shape: [torch.Size([1, 31])]
codebook_indices before unpacking: torch.Size([31])
codebook_indices after unpacking: torch.Size([1, 31])
before rearranging: semantic_token_ids.shape torch.Size([1, 31]), coarse_token_ids.shape torch.Size([1, 32, 3])
after rearranging: semantic_token_ids.shape torch.Size([1, 31]), coarse_token_ids.shape torch.Size([1, 96])
0: loss: 56.759849548339844
embed.keys(): dict_keys(['x', 'padding_mask', 'features'])
embed['x'] shape: torch.Size([1, 31, 768]), embed['features'].shape: torch.Size([1, 31, 768])
wav_input shape: torch.Size([1, 10240]), embed shape: torch.Size([31, 

In [ ]:
import numpy as np

batch_size = 1
# sample data[1] is 77040 samples at 16kHz sampling rate, 93680 for dataset[0]
# just reshape it here so batch size for prime_wave is effectively 1
samples = np.array([dataset[0]["audio"]["array"]])
prime_wave = torch.tensor(samples).reshape(len(samples), len(samples[0])).cuda() # reshape from samples to batch x num_samples
print(prime_wave.shape)
# raise AssertionError(prime_wave.shape)
max_length = 2048

semantic_tokens = semantic.generate(
    text_embeds = None, # no text, it's not musicLM
    batch_size = batch_size,
    prime_wave = prime_wave,
    max_length = max_length
)
print(f"semantic_tokens.shape {semantic_tokens.shape}")

coarse_tokens = coarse.generate(
    text_embeds = None,
    semantic_token_ids = semantic_tokens,
    reconstruct_wave = False # just want the tokens
)

torch.Size([1, 93680])
embed.keys(): dict_keys(['x', 'padding_mask', 'features'])
embed['x'] shape: torch.Size([1, 292, 768]), embed['features'].shape: torch.Size([1, 292, 768])
wav_input shape: torch.Size([1, 93680]), embed shape: torch.Size([292, 768]), packed_shape: [torch.Size([1, 292])]
codebook_indices before unpacking: torch.Size([292])
codebook_indices after unpacking: torch.Size([1, 292])
ids.shape: torch.Size([1, 292]) and prime_wave True



generating semantic:  41%|████      | 773/1895 [00:25<00:36, 30.39it/s]


before masking eos, sample_semantic_ids.shape: torch.Size([1, 927])
semantic_tokens.shape torch.Size([1, 927])



generating coarse:  22%|██▏       | 111/512 [08:42<31:28,  4.71s/it]


KeyboardInterrupt: ignored

In [6]:
!pip install torch 
import torch
x = 50 * torch.arange(12)
# x = torch.tensor([[1,2,3],[4,5,6]])
# print(x)
x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


tensor([  0,  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550])

In [5]:
!pip install einops
import einops
einops.repeat(x, 'q -> 1 (3 q)')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 3.8 MB/s eta 0:00:00


tensor([[  0,  50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550,   0,  50,
         100, 150, 200, 250, 300, 350, 400, 450, 500, 550,   0,  50, 100, 150,
         200, 250, 300, 350, 400, 450, 500, 550]])